In [1]:
!pip install transformers==4.5.0 fugashi==1.1.0 ipadic==1.0.0

In [2]:
import numpy as np
import torch
from transformers import BertJapaneseTokenizer, BertForMaskedLM

この章では文章の穴埋めを行う。

In [3]:
model_name = 'cl-tohoku/bert-base-japanese-whole-word-masking'
tokenizer = BertJapaneseTokenizer.from_pretrained(model_name)
bert_mlm = BertForMaskedLM.from_pretrained(model_name)
bert_mlm = bert_mlm.cuda()

Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTを使って穴埋めを行うには、文章の一部を特殊トークンにする。

In [4]:
text = '今日は[MASK]へ行く。'
tokens = tokenizer.tokenize(text)
print(tokens)

['今日', 'は', '[MASK]', 'へ', '行く', '。']


In [5]:
# 符号化→GPU
input_ids = tokenizer.encode(text, return_tensors = 'pt')
input_ids = input_ids.cuda()

# 入力し、出力を得る
with torch.no_grad():
    output = bert_mlm(input_ids = input_ids)
    scores = output.logits

In [6]:
scores.size()

torch.Size([1, 8, 32000])

In [7]:
# maskは4つ目にある
mask_position = input_ids[0].tolist().index(4)

# 一番スコアが高いトークンでmaskを置き換える
id_best = scores[0, mask_position].argmax(-1).item()
token_best = tokenizer.convert_ids_to_tokens(id_best)
token_best = token_best.replace('##', '')

text = text.replace('[MASK]', token_best)
print(text)

今日は東京へ行く。


In [8]:
# 上位10個を入れてみる
def predict_mask_topk(text, tokenizer, bert_mlm, num_topk):
    """
    文章中の最初の[MASK]を上位のトークンで置換
    num_topk：何位まで使うか
    """

    input_ids = tokenizer.encode(text, return_tensors = 'pt')
    input_ids = input_ids.cuda()
    with torch.no_grad():
        output = bert_mlm(input_ids = input_ids)
    scores = output.logits

    # スコアが上位のトークンとスコアを求める
    mask_position = input_ids[0].tolist().index(4)
    # https://pytorch.org/docs/stable/generated/torch.topk.html
    topk = scores[0, mask_position].topk(num_topk)
    ids_topk = topk.indices # トークンのID
    tokens_topk = tokenizer.convert_ids_to_tokens(ids_topk)
    scores_topk = topk.values.cpu().numpy()

    # 文章中の[MASK]を置換
    text_topk = []
    for token in tokens_topk:
        token = token.replace('##', '')
        text_topk.append(text.replace('[MASK]', token, 1))
    return text_topk, scores_topk

text = '今日は[MASK]へ行く。'
text_topk, score_text = predict_mask_topk(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep = '\n')

今日は東京へ行く。
今日はハワイへ行く。
今日は学校へ行く。
今日はニューヨークへ行く。
今日はどこへ行く。
今日は空港へ行く。
今日はアメリカへ行く。
今日は病院へ行く。
今日はそこへ行く。
今日はロンドンへ行く。


In [9]:
# 複数MASKがある場合の貪欲法
def greedy_prediction(text, tokenizer, bert_mlm):
    """
    [MASK]を含む文章を入力として、貪欲法で穴埋めを行った文章を出力
    """
    # 前から順にMASKを一番スコア高いやつで埋めていく
    for _ in range(text.count('[MASK]')):
        text = predict_mask_topk(text, tokenizer, bert_mlm, 1)[0][0] # 一番左のMASKを埋めてまたTEXTにする
    return text

text = '今日は[MASK][MASK]へ行く。'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は、東京へ行く。'

In [10]:
text = '今日は[MASK][MASK][MASK]'
greedy_prediction(text, tokenizer, bert_mlm)

'今日は社会科学。'

BERTは事前学習で周りの文脈からもとのトークンを予測するタスクを行なっているので、MASKだらけだときつい。

In [20]:
# ビームサーチによる穴埋め
def beam_search(text, tokenizer, bert_mlm, num_topk):
    num_mask = text.count('[MASK]')
    text_topk = [text]
    scores_topk = np.array([0])
    for i in range(num_mask):
        # 現在得られている文章に対して、最初のMASKを上位のトークンで置き換える
        text_candidates = [] # 穴埋めした結果
        score_candidates =[] # 穴埋めに使ったトークンのスコア
        for text_mask, score in zip(text_topk, scores_topk):
            text_topk_inner, scores_topk_inner = predict_mask_topk(
                text_mask, tokenizer, bert_mlm, num_topk
            )
            text_candidates.extend(text_topk_inner)
            score_candidates.append(score + scores_topk_inner)
        
        # 穴埋めで得られた文章から合計スコアの高いものを選ぶ
        score_candidates = np.hstack(score_candidates)
        ids_list = score_candidates.argsort()[::-1][:num_topk]
        text_topk = [text_candidates[ids] for ids in ids_list]
        scores_topk = score_candidates[ids_list]
        print('filled {} MASK:'.format(i+1))
        print(text_topk)
    
    return text_topk

text = '今日は[MASK][MASK]へ行く。'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep = '\n')

filled 1 MASK:
['今日は、[MASK]へ行く。', '今日は再び[MASK]へ行く。', '今日はその[MASK]へ行く。', '今日はあの[MASK]へ行く。', '今日は同じ[MASK]へ行く。', '今日はお[MASK]へ行く。', '今日はこの[MASK]へ行く。', '今日は新しい[MASK]へ行く。', '今日はゲーム[MASK]へ行く。', '今日は東京[MASK]へ行く。']
filled 2 MASK:
['今日はお台場へ行く。', '今日はお祭りへ行く。', '今日はゲームセンターへ行く。', '今日はお風呂へ行く。', '今日はゲームショップへ行く。', '今日は東京ディズニーランドへ行く。', '今日はお店へ行く。', '今日は同じ場所へ行く。', '今日はあの場所へ行く。', '今日は同じ学校へ行く。']
今日はお台場へ行く。
今日はお祭りへ行く。
今日はゲームセンターへ行く。
今日はお風呂へ行く。
今日はゲームショップへ行く。
今日は東京ディズニーランドへ行く。
今日はお店へ行く。
今日は同じ場所へ行く。
今日はあの場所へ行く。
今日は同じ学校へ行く。


In [21]:
text = '今日は[MASK][MASK][MASK]'
text_topk = beam_search(text, tokenizer, bert_mlm, 10)
print(*text_topk, sep = '\n')

filled 1 MASK:
['今日は社会[MASK][MASK]', '今日は専用[MASK][MASK]', '今日は死刑[MASK][MASK]', '今日は時代[MASK][MASK]', '今日は2[MASK][MASK]', '今日は天皇[MASK][MASK]', '今日は[UNK][MASK][MASK]', '今日は人[MASK][MASK]', '今日は1[MASK][MASK]', '今日は王[MASK][MASK]']
filled 2 MASK:
['今日は死刑廃止[MASK]', '今日は死刑囚[MASK]', '今日は社会科学[MASK]', '今日は社会学者[MASK]', '今日は社会学[MASK]', '今日は社会福祉[MASK]', '今日は死刑執行[MASK]', '今日は社会教育[MASK]', '今日は社会主義[MASK]', '今日は社会運動[MASK]']
filled 3 MASK:
['今日は社会学者。', '今日は死刑廃止。', '今日は社会学。', '今日は社会科学。', '今日は死刑廃止派', '今日は死刑囚。', '今日は社会福祉法人', '今日は社会主義。', '今日は死刑廃止運動', '今日は社会運動。']
今日は社会学者。
今日は死刑廃止。
今日は社会学。
今日は社会科学。
今日は死刑廃止派
今日は死刑囚。
今日は社会福祉法人
今日は社会主義。
今日は死刑廃止運動
今日は社会運動。
